In [2]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

## Bootstrapping exercise
- There are two correlated stocks, $X$ and $Y$, which are normally distributed
- You will invest in both stocks with all your money
- Your goal is to find $\alpha \in [0,1]$ (the portion of your money invested in stock $X$) to minimize the variance of the portfolio, i.e.,
 $$ \mathbb{V}\mathrm{AR}[\alpha X + (1-\alpha)Y] $$
- You are given "returns.npy", a 2-D numpy array of the past $200$ returns of $(X,Y)$.

Exercise questions: 
1. Estimate $\mu_X, \mu_Y, \sigma_{X}^2, \sigma_{Y}^2, \sigma_{XY}$ from the sample, and then compute the optimal investment strategy $\alpha$
2. Since we estimated the parameters by looking at a sample, we may be severly biased. Hence, use bootstrapping to iteratively sample $200$ points (with replacement) from the previous returns, and estimate the optimal investment strategy each time. Apply this $B = 500$ times and compute the standard error of the optimal $\alpha$ that we computed on the original set of previous returns.

In [3]:
# load
returns = np.load('returns.npy')
n = 200

In [4]:
#Estimate mean returns
hat_mean = np.mean(returns,0)
print(hat_mean)

[2.97259589 2.98935547]


In [5]:
#estimate covariance matrix
hat_cov = np.zeros((2,2))
for i in range(n):
    hat_cov = hat_cov + (returns[i]-hat_mean).reshape((2,1))*(returns[i]-hat_mean).reshape((1,2))
hat_cov = hat_cov/(n-1)
hat_varx = hat_cov[0,0]
hat_vary = hat_cov[1,1]
hat_covar = hat_cov[0,1]
print(hat_varx)
print(hat_vary)
print(hat_covar)

1.291266848642483
1.1075660892225494
0.4650946600840741


Optimal $\alpha^\star$ -
\begin{align*}
\alpha^\star = \dfrac{\sigma^2_Y - \sigma_{XY}}{\sigma^2_X + \sigma^2_Y - 2\sigma_{XY}}.
\end{align*}

In [6]:
#estimate optimal investment
def optimal_alpha(varx, vary, covar):
    return (float) (vary - covar)/(varx + vary - 2*covar)
optimal_investment = round(optimal_alpha(hat_varx, hat_vary, hat_covar),3)
print(optimal_investment)

0.437


In [10]:
#use bootstrapping for std. error associated with this technique of finding the optimal investment
B = 500 #number of bootstraps
samples = n #sample n each time
simulation = 1000
estimations = np.zeros(simulation) #save estimated alphas

def generate_sample(n, varx=hat_varx, vary=hat_vary, covar=hat_covar): #generate a sample of n returns
    mu = np.array([2 ,2])
    r = np.array([
            [  varx, covar],
            [ covar,  vary]])
    sample_returns = np.random.multivariate_normal(mu, r, size=n)
    return sample_returns

def hat_alpha(sample_returns): 
    hat_mean = np.mean(sample_returns,0)
    hat_cov = np.zeros((2,2))
    n = np.size(sample_returns,0)
    for i in range(n):
        hat_cov = hat_cov + (sample_returns[i]-hat_mean).reshape((2,1))*(sample_returns[i]-hat_mean).reshape((1,2))
    hat_cov = hat_cov/(n-1)
    hat_varx = hat_cov[0,0]
    hat_vary = hat_cov[1,1]
    hat_covar = hat_cov[0,1]
    return round(optimal_alpha(hat_varx, hat_vary, hat_covar),3)

for sim in range(simulation): #simulate this many times
    generated_sample = generate_sample(samples) #sample with replacements
    estimations[sim] = hat_alpha(generated_sample) #write the optimized alpha on the sample
bootstrap_error = np.sqrt( np.sum(np.square(estimations - bar_alpha)) /(B-1))
round(bootstrap_error,3)

0.077

Report `optimal_investment` and `bootrap_error`